# FastAPI ile Modern API Geliştirme: Kapsamlı Rehber

## 🚀 Bölüm 1: Framework Karşılaştırması

Python'da web geliştirme için birçok harika araç bulunur. En popüler üç tanesini karşılaştıralım:

| Özellik | Flask | Django | FastAPI |
|---|---|---|---|
| **Felsefe** | Minimalist, esnek, "micro-framework" | "Batteries-included", her şey dahil, katı | Modern, yüksek performans, standartlara dayalı |
| **Güçlü Yönleri** | Basitlik, küçük projeler, esneklik | Hızlı geliştirme, ORM, Admin paneli | Asenkron, hız, Pydantic, otomatik dokümantasyon |
| **Zayıf Yönleri** | Büyük projelerde yapı kurma sorumluluğu kullanıcıda | Monolitik yapı, daha az esnek | Ekosistemi Django kadar geniş değil |
| **Kullanım Alanı** | Küçük API'ler, basit web siteleri, prototipler | Büyük ve karmaşık web uygulamaları, CMS | Yüksek performanslı API'ler, mikroservisler |

**Sonuç:** Eğer amacınız modern, hızlı ve ölçeklenebilir bir API oluşturmaksa, **FastAPI** asenkron yapısı ve Pydantic entegrasyonu ile en iyi seçenektir.

---

## 🛠️ Bölüm 2: Temel Kavramlar

### 2.1 HTTP Metotları
API'ler, istemci ve sunucu arasındaki iletişimi standart HTTP metotları ile sağlar. En temelleri: `GET`, `POST`, `PUT`, `DELETE`.

### 2.2 Pydantic ile Veri Doğrulama
FastAPI'nin gücü, Pydantic'ten gelir. Gelen isteklerin gövdesini (body) otomatik olarak doğrular, hataları yakalar ve veriyi Python nesnelerine dönüştürür.

In [4]:
from pydantic import BaseModel, Field

class Book(BaseModel):
    title: str = Field(..., min_length=3, description="Kitap başlığı")
    author: str
    publication_year: int | None = Field(default=None, gt=1400)

# Bu model, bir POST veya PUT isteği geldiğinde FastAPI tarafından otomatik olarak kullanılacaktır.
# Eğer gelen veri bu şemaya uymazsa, FastAPI otomatik olarak 422 Unprocessable Entity hatası döndürür.

### 2.3 Durum Kodlarını Sınıflarla Yönetme
API'de standart durum kodları kullanmak önemlidir. Bu kodları merkezi bir yerden yönetmek, kod okunabilirliğini artırır. `Enum` bunun için harika bir yoldur.

In [5]:
from enum import IntEnum

class HTTPStatusCodes(IntEnum):
    OK = 200
    CREATED = 201
    NO_CONTENT = 204
    BAD_REQUEST = 400
    UNAUTHORIZED = 401
    FORBIDDEN = 403
    NOT_FOUND = 404
    UNPROCESSABLE_ENTITY = 422

# Kullanımı:
# from fastapi import status
# @app.post("/items", status_code=status.HTTP_201_CREATED)
# FastAPI'nin kendi 'status' modülü zaten bu yapıyı sunar, ancak kendi özel kodlarınızı bu şekilde yönetebilirsiniz.

---

## ⚡ Bölüm 3: Asenkron (Async) Operasyonlar

FastAPI, Python'un `asyncio` kütüphanesi üzerine kurulmuştur. Bu, API'nizin aynı anda birden çok isteği bekleme yapmadan (non-blocking) yönetmesini sağlar. Özellikle veritabanı sorguları veya harici API çağrıları gibi I/O-bound (Giriş/Çıkış'a bağlı) işlemler için performansı ciddi şekilde artırır.

`async def` ile tanımlanan path operasyonları, `await` ile çağrılan asenkron fonksiyonları beklerken diğer istekleri işlemeye devam edebilir.

In [6]:
import asyncio
from fastapi import FastAPI

app = FastAPI()

async def slow_db_call():
    """Veritabanı veya harici API çağrısını simüle eder."""
    await asyncio.sleep(1) # 1 saniye bekle
    return {"status": "done"}

@app.get("/slow-endpoint")
async def handle_slow_request():
    print("İstek alındı, yavaş işlem bekleniyor...")
    result = await slow_db_call()
    print("Yavaş işlem tamamlandı, yanıt gönderiliyor.")
    return result

# Bu endpoint'e birden çok istek atıldığında, FastAPI hepsini neredeyse aynı anda kabul eder
# ve her biri kendi bekleme süresini tamamlarken diğerlerini engellemez.

---

## 📥 Bölüm 4: Gelişmiş Giriş (Input) Yönetimi

FastAPI, farklı türde parametreleri kolayca yönetmenizi sağlar:

- **Path Parametreleri:** URL yolunun bir parçasıdır. Örn: `/items/{item_id}`
- **Query Parametreleri:** URL'nin sonuna `?` ile eklenir. Örn: `/items?skip=0&limit=10`
- **Body Parametreleri:** `POST`, `PUT` gibi isteklerin gövdesinde gönderilen JSON verisidir.

In [7]:
from fastapi import Body, Query, Path
from typing import Annotated

@app.put("/books/{book_id}")
async def update_book(
    # Path parametresi ve validasyonu
    book_id: Annotated[int, Path(title="Kitap ID'si", ge=1)],
    # Body'den gelecek Pydantic modeli
    book: Book,
    # Query parametresi ve validasyonu
    version: Annotated[int | None, Query(title="Versiyon Numarası", ge=1)] = None
):
    return {
        "book_id": book_id,
        "version": version,
        "updated_book_data": book.model_dump()
    }

---

## ✅ Bölüm 5: Test Otomasyonu

API geliştirmenin en önemli adımlarından biri testtir. FastAPI, `TestClient` sayesinde API'nizi test etmeyi çok kolaylaştırır.

**Sistem Testi vs. Fonksiyonel Test**
- **Fonksiyonel Test:** Bir özelliğin (örn: kitap ekleme) beklendiği gibi çalışıp çalışmadığını doğrular. Genellikle tek bir API endpoint'ini test eder.
- **Sistem Testi:** Tüm sistemin bir bütün olarak nasıl çalıştığını test eder. Birden fazla özelliğin birbiriyle etkileşimini (örn: kitap ekle, sonra listele, sonra sil) içerir.

In [8]:
# test_main.py adında bir dosyada olmalı
from fastapi.testclient import TestClient
from fastapi import status

# from main import app # Ana uygulama dosyanızdan app'i import edin

client = TestClient(app)

def test_create_read_update_delete_book(): # Bu bir sistem testine örnektir
    # 1. CREATE (POST)
    new_book_data = {"title": "Test Kitabı", "author": "Test Yazar"}
    response = client.post("/books/", json=new_book_data)
    assert response.status_code == status.HTTP_201_CREATED
    created_book = response.json()
    assert created_book["title"] == new_book_data["title"]
    book_id = created_book["id"]

    # 2. READ (GET)
    response = client.get(f"/books/{book_id}")
    assert response.status_code == status.HTTP_200_OK
    assert response.json()["title"] == new_book_data["title"]

    # 3. UPDATE (PUT)
    updated_data = {"title": "Güncellenmiş Test Kitabı", "author": "Güncellenmiş Yazar"}
    response = client.put(f"/books/{book_id}", json=updated_data)
    assert response.status_code == status.HTTP_200_OK
    assert response.json()["title"] == updated_data["title"]

    # 4. DELETE (DELETE)
    response = client.delete(f"/books/{book_id}")
    assert response.status_code == status.HTTP_204_NO_CONTENT

    # 5. Verify Deletion (GET)
    response = client.get(f"/books/{book_id}")
    assert response.status_code == status.HTTP_404_NOT_FOUND

---

## 🔐 Bölüm 6: Güvenlik ve Erişim Kontrolü

### 6.1 API Key ile Kimlik Doğrulama
API'nizi korumanın en temel yollarından biri API Key kullanmaktır. FastAPI'nin **Dependency Injection** (Bağımlılık Enjeksiyonu) sistemi bunu çok şık bir şekilde çözer.

In [9]:
from fastapi import Security, Depends, HTTPException
from fastapi.security import APIKeyHeader

API_KEY = "SECRET_API_KEY_12345"
API_KEY_NAME = "X-API-Key"

api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=False)

async def get_api_key(api_key: str = Security(api_key_header)):
    if api_key == API_KEY:
        return api_key
    else:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Could not validate credentials"
        )

# Bu fonksiyon, API anahtarını doğrulamak için kullanılacak. Eğer anahtar geçerli değilse, 403 Forbidden hatası döndürecek.
# Eğer anahtar geçerliyse, istek devam edecek ve endpoint'e erişim sağlanacak.
@app.get("/secure")
async def secure_endpoint(api_key: str = Depends(get_api_key)):
    return {"message": "Eğer bu mesajı görüyorsan, API anahtarın geçerli!"}

### 6.2 Rate Limiting (İstek Sınırlandırma)
API'nizin kötüye kullanımını veya aşırı yüklenmesini önlemek için bir istemcinin belirli bir sürede yapabileceği istek sayısını sınırlamanız gerekir. Bu genellikle `slowapi` gibi harici kütüphanelerle yapılır.

In [10]:
# Örnek Kurulum
# pip install slowapi

from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded

limiter = Limiter(key_func=get_remote_address)
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

from fastapi import Request

@app.get("/limited")
@limiter.limit("5/minute") # Dakikada 5 istek
async def limited_endpoint(request: Request): # request: Request parametresi gerekir
    return {"message": "Bu endpoint'in bir limiti var."}

---

## Advanced Bölüm 7: İleri Düzey Konular

### 7.1 Asenkron Arka Plan Görevleri (Background Tasks)
Bazen bir isteğe hemen yanıt dönmek, ama arka planda uzun süren bir işlemi (örn: e-posta gönderme, rapor oluşturma) başlatmak istersiniz. `BackgroundTasks` bunun için idealdir.

In [11]:
from fastapi import BackgroundTasks

def write_notification(email: str, message=""):
    with open("log.txt", mode="a") as email_file:
        content = f"notification for {email}: {message}\n"
        email_file.write(content)

@app.post("/send-notification/{email}")
async def send_notification(email: str, background_tasks: BackgroundTasks):
    background_tasks.add_task(write_notification, email, message="some notification")
    return {"message": "Notification sent in the background"}

### 7.2 API Versiyonlama
API'niz geliştikçe, mevcut istemcileri bozmadan değişiklikler yapmanız gerekir. En yaygın yöntem URL tabanlı versiyonlamadır:
`/api/v1/items`
`/api/v2/items`

Bunu FastAPI'de `APIRouter` veya Sub-Applications kullanarak yönetebilirsiniz.

### 7.3 Otomatik Dokümantasyon
FastAPI'nin en sevilen özelliklerinden biri, kodunuzdan otomatik olarak interaktif dokümantasyon oluşturmasıdır. Sunucunuz çalışırken şu adreslere gidin:
- **/docs**: Swagger UI arayüzü.
- **/redoc**: ReDoc arayüzü.

---

## 📈 Bölüm 8: Prodüksiyon (Production) Hazırlığı

### 8.1 Monitoring (İzleme) ve Logging (Kayıt Tutma)
Bir API'yi canlıya aldıktan sonra onun sağlığını ve performansını izlemek kritik öneme sahiptir.

**Logging:** Python'un `logging` modülü, uygulamanızda olan bitenleri (hatalar, önemli olaylar, istek bilgileri) bir dosyaya veya servise kaydetmenizi sağlar. Bu, sorunları teşhis etmek için paha biçilmezdir.

**Monitoring:** Prometheus, Grafana, Datadog gibi araçlar, API'nizin yanıt süreleri, hata oranları, kaynak kullanımı gibi metriklerini görselleştirerek genel durumu hakkında size bilgi verir. FastAPI uygulamaları, bu araçlarla entegre olmak için `starlette-prometheus` gibi kütüphaneleri kullanabilir.

# Örnek FastAPI uygulaması

In [12]:
"""
FastAPI ile Modern API Geliştirme - Kapsamlı Uygulama
Bu dosya, FastAPI notebook'undaki tüm kod örneklerini içerir.
"""

import asyncio
from enum import IntEnum
from typing import Annotated
import logging
from contextlib import asynccontextmanager
from fastapi import (
    FastAPI,
    Body,
    Query,
    Path,
    Security,
    Depends,
    HTTPException,
    BackgroundTasks,
    status,
    Request
)
from fastapi.security import APIKeyHeader
from pydantic import BaseModel, Field
from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded

# Logging configuration
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ===== Rate Limiter Configuration =====
limiter = Limiter(key_func=get_remote_address)

# ===== In-Memory Database (for demo purposes) =====
books_db = []
book_id_counter = 1

# ===== Lifespan Event Handler =====
@asynccontextmanager
async def lifespan(app: FastAPI):
    """Handle application startup and shutdown events."""
    # Startup
    logger.info("FastAPI application starting up...")

    # Add some sample data
    sample_books = [
        {"id": 1, "title": "The Hobbit", "author": "J.R.R. Tolkien", "publication_year": 1937},
        {"id": 2, "title": "1984", "author": "George Orwell", "publication_year": 1949},
        {"id": 3, "title": "Dune", "author": "Frank Herbert", "publication_year": 1965}
    ]

    books_db.extend(sample_books)
    global book_id_counter
    book_id_counter = 4

    logger.info(f"Loaded {len(sample_books)} sample books")

    yield  # Application runs here

    # Shutdown
    logger.info("FastAPI application shutting down...")

# FastAPI app instance with lifespan
app = FastAPI(
    title="Library Management API",
    description="A comprehensive FastAPI example with all features demonstrated",
    version="1.0.0",
    lifespan=lifespan
)

# Add rate limiter to app
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

# ===== Pydantic Models =====
class Book(BaseModel):
    """Book model with Pydantic validation."""
    title: str = Field(..., min_length=3, description="Kitap başlığı")
    author: str
    publication_year: int | None = Field(default=None, gt=1400)

class BookCreate(BaseModel):
    """Model for creating a new book."""
    title: str = Field(..., min_length=3)
    author: str
    publication_year: int | None = Field(default=None, gt=1400)

class BookResponse(BaseModel):
    """Model for book response with ID."""
    id: int
    title: str
    author: str
    publication_year: int | None = None

# ===== HTTP Status Codes =====
class HTTPStatusCodes(IntEnum):
    """Custom HTTP status codes enum."""
    OK = 200
    CREATED = 201
    NO_CONTENT = 204
    BAD_REQUEST = 400
    UNAUTHORIZED = 401
    FORBIDDEN = 403
    NOT_FOUND = 404
    UNPROCESSABLE_ENTITY = 422

# ===== Security Configuration =====
API_KEY = "SECRET_API_KEY_12345"
API_KEY_NAME = "X-API-Key"

api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=False)

async def get_api_key(api_key: str = Security(api_key_header)):
    """Validate API key for secured endpoints."""
    if api_key == API_KEY:
        return api_key
    else:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Could not validate credentials"
        )

# ===== Async Helper Functions =====
async def slow_db_call():
    """Simulate a slow database or external API call."""
    await asyncio.sleep(1)  # 1 saniye bekle
    return {"status": "done"}

# ===== Background Task Functions =====
def write_notification(email: str, message=""):
    """Write notification to log file."""
    try:
        with open("log.txt", mode="a") as email_file:
            content = f"notification for {email}: {message}\n"
            email_file.write(content)
        logger.info(f"Notification written for {email}")
    except Exception as e:
        logger.error(f"Failed to write notification: {e}")

# ===== API Endpoints =====

@app.get("/")
async def root():
    """Root endpoint with welcome message."""
    return {"message": "FastAPI Library Management System"}

@app.get("/health")
async def health_check():
    """Health check endpoint."""
    return {"status": "healthy", "timestamp": "2025-07-30"}

# ===== Async Endpoint Example =====
@app.get("/slow-endpoint")
async def handle_slow_request():
    """Demonstrate async operations."""
    logger.info("İstek alındı, yavaş işlem bekleniyor...")
    result = await slow_db_call()
    logger.info("Yavaş işlem tamamlandı, yanıt gönderiliyor.")
    return result

# ===== Book Management Endpoints =====
@app.post("/books/", response_model=BookResponse, status_code=status.HTTP_201_CREATED)
async def create_book(book: BookCreate):
    """Create a new book."""
    global book_id_counter

    new_book = BookResponse(
        id=book_id_counter,
        title=book.title,
        author=book.author,
        publication_year=book.publication_year
    )

    books_db.append(new_book.model_dump())
    book_id_counter += 1

    logger.info(f"Created book: {new_book.title}")
    return new_book

@app.get("/books/", response_model=list[BookResponse])
async def list_books(
    skip: Annotated[int, Query(description="Number of books to skip", ge=0)] = 0,
    limit: Annotated[int, Query(description="Number of books to return", ge=1, le=100)] = 10
):
    """List all books with pagination."""
    return books_db[skip:skip + limit]

@app.get("/books/{book_id}", response_model=BookResponse)
async def get_book(
    book_id: Annotated[int, Path(title="Book ID", ge=1)]
):
    """Get a specific book by ID."""
    for book in books_db:
        if book["id"] == book_id:
            return book

    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Book with ID {book_id} not found"
    )

@app.put("/books/{book_id}", response_model=BookResponse)
async def update_book(
    book_id: Annotated[int, Path(title="Kitap ID'si", ge=1)],
    book: Book,
    version: Annotated[int | None, Query(title="Versiyon Numarası", ge=1)] = None
):
    """Update a book with advanced parameter handling."""
    for i, existing_book in enumerate(books_db):
        if existing_book["id"] == book_id:
            updated_book = {
                "id": book_id,
                "title": book.title,
                "author": book.author,
                "publication_year": book.publication_year
            }
            books_db[i] = updated_book

            logger.info(f"Updated book {book_id}, version: {version}")
            return updated_book

    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Book with ID {book_id} not found"
    )

@app.delete("/books/{book_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_book(
    book_id: Annotated[int, Path(title="Book ID", ge=1)]
):
    """Delete a book by ID."""
    for i, book in enumerate(books_db):
        if book["id"] == book_id:
            deleted_book = books_db.pop(i)
            logger.info(f"Deleted book: {deleted_book['title']}")
            return

    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Book with ID {book_id} not found"
    )

# ===== Secured Endpoints =====
@app.get("/secure")
async def secure_endpoint(api_key: str = Depends(get_api_key)):
    """Secured endpoint requiring API key."""
    return {"message": "Eğer bu mesajı görüyorsan, API anahtarın geçerli!"}

@app.get("/secure/books", response_model=list[BookResponse])
async def secure_list_books(api_key: str = Depends(get_api_key)):
    """Secured endpoint to list books."""
    return books_db

# ===== Background Tasks Endpoint =====
@app.post("/send-notification/{email}")
async def send_notification(
    email: str,
    background_tasks: BackgroundTasks,
    message: str = "Book notification"
):
    """Send notification in background."""
    background_tasks.add_task(write_notification, email, message=message)
    return {"message": "Notification sent in the background"}

# ===== Rate Limited Endpoints =====
@app.get("/limited")
@limiter.limit("5/minute")
async def limited_endpoint(request: Request):
    """
    Rate limited endpoint - maximum 5 requests per minute per IP.
    """
    return {"message": "Bu endpoint dakikada maksimum 5 istek kabul eder."}

@app.get("/limited-strict")
@limiter.limit("2/minute")
async def strict_limited_endpoint(request: Request):
    """
    Strict rate limited endpoint - maximum 2 requests per minute per IP.
    """
    return {"message": "Bu endpoint dakikada maksimum 2 istek kabul eder."}

@app.get("/limited-books")
@limiter.limit("10/minute")
async def rate_limited_books(request: Request):
    """
    Rate limited books endpoint - maximum 10 requests per minute per IP.
    """
    return {
        "message": "Rate limited books endpoint",
        "total_books": len(books_db),
        "books": books_db[:5]  # Return only first 5 books
    }

# ===== API Version Examples =====
@app.get("/api/v1/books", response_model=list[BookResponse])
async def list_books_v1():
    """Version 1 of books endpoint."""
    return books_db

@app.get("/api/v2/books")
async def list_books_v2():
    """Version 2 of books endpoint with enhanced response."""
    return {
        "version": "2.0",
        "total_books": len(books_db),
        "books": books_db
    }

# ===== Error Handling Examples =====
@app.get("/error-demo")
async def error_demo():
    """Demonstrate error handling."""
    raise HTTPException(
        status_code=status.HTTP_400_BAD_REQUEST,
        detail="This is a demo error"
    )

if __name__ == "__main__":
    import uvicorn

    # Run the application
    uvicorn.run(
        "fastapi_main:app",
        host="127.0.0.1",
        port=8000,
        reload=True,
        log_level="info"
    )

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [466] using StatReload
INFO:     Stopping reloader process [466]
